<a href="https://colab.research.google.com/github/elainedias16/TCC/blob/main/Next_step_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
pip install transformers torch

Architeture
https://dugas.ch/artificial_curiosity/GPT_architecture.html

https://keras.io/examples/generative/text_generation_with_miniature_gpt/

https://huggingface.co/learn/nlp-course/chapter7/6

https://debuggercafe.com/text-generation-with-transformers/0

https://debuggercafe.com/text-generation-with-transformers/

## Masked Self-Attention

Scale dot produt attetion:
https://paperswithcode.com/method/scaled

In [139]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.query = nn.Linear(config.d_model, config.head_dim, bias=config.bias)
    self.key = nn.Linear(config.d_model, config.head_dim, bias=config.bias)
    self.value = nn.Linear(config.d_model, config.head_dim, bias=config.bias)


  def forward(self, x):
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    return q, k, v



class MaskedSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads
        self.head_dim = config.head_dim
        self.dropout = nn.Dropout(config.dropout)
        self.d_model = config.d_model
        self.heads = nn.ModuleList([Head(config) for _ in range(config.num_heads)])
        self.output_linear = nn.Linear(config.d_model, config.d_model)
        assert self.head_dim * self.num_heads == self.d_model, "d_model must be divisible by num_heads"


    def forward(self, x, mask=None):
        # B, T, C = x.size()

        heads_output = []
        for head in self.heads:
            k, q, v = head(x)


            # Scaled dot-product attention
            scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

            if mask is not None:
                scores = scores.masked_fill(mask == 0, float('-inf'))

            attn_weights = F.softmax(scores, dim=-1)
            attn_weights = self.dropout(attn_weights)

            head_output = torch.matmul(attn_weights, v)
            heads_output.append(head_output)


        concatenated_output = torch.cat(heads_output, dim=-1)
        output = self.output_linear(concatenated_output)

        return output




## Feed Forward Nerual Network


MLP

In [140]:
class FeedFoward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear1 = nn.Linear(config.d_model, 4 * config.d_model, bias=config.bias)
    self.activation = nn.ReLU()
    self.linear2 = nn.Linear(config.d_model * 4,  config.d_model, bias=config.bias)
    self.dropout = nn.Dropout(config.dropout)


  def forward(self, x):
    x = self.linear1(x)
    x = self.activation(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x



## Layer Norm

In [141]:
class LayerNorm(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.norm = nn.LayerNorm(config.d_model, config.bias)

  def forward(self, x):
    self.norm(x)
    return x

## One Decoder

In [142]:
class Decoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.ln_1 = LayerNorm(config)
    self.masked_self_attention = MaskedSelfAttention(config)
    self.ln_2 = LayerNorm(config)
    self.feed_forward = FeedFoward(config)

  # def forward(self, x, mask):
  #   x = self.ln_1(x)
  #   x = x + self.masked_self_attention(x, mask)
  #   x = self.ln_2(x)
  #   x = x + self.feed_forward(x)
  #   return x
  def forward(self, x):
    x = self.ln_1(x)
    x = x + self.masked_self_attention(x)
    x = self.ln_2(x)
    x = x + self.feed_forward(x)
    return x



## Transformer

https://medium.com/@hunter-j-phillips/positional-encoding-7a93db4109e6#:~:text=class%20PositionalEncoding(nn,self.dropout(x)
olhar posiitional

In [143]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.word_token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_embedding = nn.Embedding(config.max_length, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        self.blocks = nn.Sequential(*[Decoder(config) for _ in range(config.n_layer)])
        self.ln = LayerNorm(config)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

    def forward(self, input_ids, targets=None):
        device = input_ids.device
        B, T = input_ids.size()

        # Positional e token embed
        tok_emb = self.word_token_embedding(input_ids)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = self.dropout(tok_emb + pos_emb)

        # Transformer blocks
        x = self.blocks(x)

        # Norm layer
        x = self.ln(x)

        # Final layer
        logits = self.lm_head(x)

        if targets is not None:
            # Shift logits and targets to align
            shift_logits = logits[:, :-1, :].contiguous()
            shift_targets = targets[:, 1:].contiguous()
            loss = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_targets.view(-1))
            return logits, loss

        return logits, None

    def generate(self, input_ids, max_new_tokens):
        new_tokens = []

        for _ in range(0, max_new_tokens):
            input_ids_cond = input_ids[:, -self.config.block_size:]
            logits, _ = self.forward(input_ids_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            input_ids_next = torch.multinomial(probs, num_samples=1)
            new_tokens.append(input_ids_next)
            input_ids = torch.cat((input_ids, input_ids_next), dim=1)

        new_tokens = torch.cat(new_tokens, dim=1)
        return new_tokens


## Config

In [152]:
import torch

#Paramters:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_new_tokens = 50
epochs = 100
learning_rate = 0.001
batch_size = 8
SEQUENCE_LENGTH = 64


class Config:
    num_heads = 2
    d_model = 8 #os vetores de entrada e saída terão dimensão 8
    head_dim = 4 #cada cabeça tem dimensão 4
    dropout = 0.1  #para evitar overfiting
    bias = True
    vocab_size = 50257  # len tokenizer
    # hidden_size = 1024
    max_length = 512
    n_layer = 6
    # block_size = 1024
    # block_size = 32
    block_size = SEQUENCE_LENGTH


config = Config()

## Training

In [145]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Dataset Preparation

In [146]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

--2024-07-02 01:15:42--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.7’

input.txt.7         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-07-02 01:15:42 (17.4 MB/s) - ‘input.txt.7’ saved [1115394/1115394]



In [147]:
class TextDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [148]:
def tokenize(text):
  return word_tokenize(text.lower())


def build_vocab(tokens):
  word_counts = Counter(tokens)
  vocab = list(word_counts.keys())
  word_to_int = {word: i for i, word in enumerate(vocab)}
  int_to_word = {i: word for word, i in word_to_int.items()}
  return len(vocab), word_to_int, int_to_word



def prepare_data(text, word_to_int, sequence_length=64):
  # Tokenize the text
  tokens = tokenize(text)
  # Encode the tokens into integers
  encoded = [word_to_int[token] for token in tokens]
  # Create training samples
  samples = [encoded[i:i + sequence_length + 1] for i in range(len(encoded) - sequence_length)]
  # Separate input and target sequences
  input_ids = torch.tensor([sample[:-1] for sample in samples], dtype=torch.long)
  targets = torch.tensor([sample[1:] for sample in samples], dtype=torch.long)
  return input_ids, targets


def tokenize_and_encode(text, word_to_int):
    tokens = tokenize(text)
    # Add a check if '<pad>' is in the vocabulary
    if '<pad>' not in word_to_int:
        # If not, assign it a default value (e.g., 0)
        pad_id = 0
    else:
        pad_id = word_to_int['<pad>']
    encoded = [word_to_int.get(token, pad_id) for token in tokens]
    return torch.tensor(encoded, dtype=torch.long).unsqueeze(0)


def decode_tokens(tokens, int_to_word):
    return ' '.join([int_to_word[token.item()] for token in tokens.squeeze()])


def generate_text(model, start_text, word_to_int, int_to_word, max_new_tokens=50):
    model.eval()
    with torch.no_grad():
        input_ids = tokenize_and_encode(start_text, word_to_int)
        generated_tokens = model.generate(input_ids, max_new_tokens)
        return decode_tokens(generated_tokens, int_to_word)


tokenize_text = tokenize(text)
vocab_size, word_to_int, int_to_word = build_vocab(tokenize_text)

##Train

## Run

In [149]:
model = Transformer(config)
optimizer = Adam(model.parameters(), lr=learning_rate)

In [151]:
with open('alice_1.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# text = "Alice was a curious and imaginative young girl"

def train_model(model, dataloader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for input_ids, targets in dataloader:
            optimizer.zero_grad()
            logits, loss = model(input_ids, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


tokens = tokenize(text)
config.vocab_size, word_to_int, int_to_word = build_vocab(tokens)
print(f"vocab_size: {config.vocab_size}")
input_ids, targets = prepare_data(text, word_to_int)
print(f"input_ids shape: {input_ids.shape}")
print(f"targets shape: {targets.shape}")

dataset = TextDataset(input_ids, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = Transformer(config)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train_model(model, dataloader, optimizer, epochs=epochs)

start_text = "Alice was walking through the forest when she suddenly saw a strange"
generated_text = generate_text(model, start_text, word_to_int, int_to_word, max_new_tokens=max_new_tokens)
print(generated_text)


####################Test##################################
# test_sentences = [
#     "First Citizen: Before we proceed any further, hear me speak.",
#     "All: Speak, speak.",
#     "First Citizen: You are all resolved rather to die than to famish?",
#     "All: Resolved. resolved.",
#     "First Citizen: First, you know Caius Marcius is chief enemy to the people.",
#     "All: We know't, we know't."
# ]

test_sentences = [
    "Alice was walking through the forest when she suddenly saw a strange.",
    "As Alice pleaded with the Queen for help",
    "Continuing her exploration, Alice encountered ",
    "Alice delved into books",
    "First Citizen: First, you know Caius Marcius is chief enemy to the people.",
    "Alice began to explore lucid dreaming"
]

for sentence in test_sentences:
    generated_text = generate_text(model, sentence, word_to_int, int_to_word, max_new_tokens=50)
    print(f"Input: {sentence}")
    print(f"Generated: {generated_text}")
    print("="*80)

vocab_size: 375
input_ids shape: torch.Size([789, 64])
targets shape: torch.Size([789, 64])
Epoch 1, Loss: 5.836219368558941
Epoch 2, Loss: 5.084947489728831
follow continuing enough that began that after , . offered distort trial , rabbit determined was her she stumbled home , unpredictable . knave including alice a home realized claimed the her taller courage and bolder familiar confidence nearby as . sunny , awakening off of cat , day with
Input: Alice was walking through the forest when she suddenly saw a strange.
Generated: she the . quiet curls embraced courtyard revelations 's found eccentric and with colors could , after hoping of the hole were an sage that reminder approached 's through led rabbit , and home out , queen , taller dream and tale using her confidence mad however shrink myths when
Input: As Alice pleaded with the Queen for help
Generated: second into the fell she sunny rabbit palace individuals a queen determined with stuck of size logic will best , own determined

In [ ]:
test_sentence = "How are you?"
generated_text = generate_text(model, sentence, word_to_int, int_to_word, max_new_tokens=50)
generated_text